In [1]:
# DEPENDENCIES
import glob
#from shutil import copyfile
import numpy as np
import pandas as pd
#import logomaker
# LOCAL IMPORTS
#from indexing_classes import GPCRDBIndexing
import sse_func
import matplotlib.pyplot as plt
import template_finder as tf

def find_pdb(name, pdb_folder):
    identifier = name.split("-")[0]
    target_pdb = glob.glob(f"{pdb_folder}/*{identifier}*.pdb")[0]
    return target_pdb

In [2]:
valid_collection = pd.read_pickle("../valid_collection.pkl")

In [3]:
for i, gain in enumerate(valid_collection.collection[93:94]):
    #if "A0A6J3E5X4" not in gain.name:
    #    continue
    #print(i)
    file_prefix = f"../test_stal_indexing/t_all_{i}"
    print("_"*30, f"\n{i} {gain.name}")
    #for i, hel in enumerate(gain.sda_helices):
    #    print(f"H#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    #for i, hel in enumerate(gain.sdb_sheets):
    #    print(f"S#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    element_intervals, element_centers, residue_labels, unindexed_elements = tf.assign_indexing(gain, 
                                                                                                file_prefix=file_prefix, 
                                                                                                gain_pdb=find_pdb(gain.name, '../all_pdbs'), 
                                                                                                template_dir='../r2_template_pdbs/', 
                                                                                                debug=True, 
                                                                                                create_pdb=False)
    #print(element_intervals, element_centers, residue_labels, unindexed_elements, sep="\n")

______________________________ 
0 A0A6A4VP58-A0A6A4VP58_AMPAM-CadherinEGFLAGseven-passG-typereceptor3-Amphibalanus_amphitrite
[DEBUG] assign_indexing: gain_obj.start = 229
	gain_obj.end = 640
	gain_obj.subdomain_boundary = 426
	gain_pdb = '../all_pdbs/A0A6A4VP58.pdb'
Looking up best template for agpcr_type = 'C3': best_a = 'C' best_b = 'E5b'
[DEBUG] assign_indexing: Found receptor type and best corresponding templates:
	agpcr_type = 'C3'
	best_a = 'C'
	best_b = 'E5b'
	a_centers = {'H1': 463, 'H2': 484, 'H3': 498, 'H4': 541, 'H5': 549, 'H6': 567}
	b_centers = {'S1': 324, 'S2': 333, 'S3': 350, 'S4': 357, 'S5': 375, 'S6': 407, 'S7': 414, 'S8': 431, 'S9': 450, 'S10': 459, 'S11': 464, 'S12': 476, 'S13': 487}
[DEBUG]: find_anchor_matches: file = '../test_stal_indexing/t_all_0_sda.out', parsing_dict = {454: (247, 4.0), 455: (None, None), 456: (248, 4.06), 457: (249, 3.22), 458: (250, 3.08), 459: (251, 1.58), 460: (252, 0.83), 461: (253, 2.21), 462: (254, 0.83), 463: (255, 2.04), 464: (256, 3.

KeyError: 541

In [ ]:
print(find_pdb("A0A6A4VP58", '../all_pdbs'))
print(gain.sequence[353-gain.start:395-gain.start])

../all_pdbs/A0A6A4VP58.pdb
['E' 'Q' 'E' 'L' 'N' 'T' 'F' 'M' 'N' 'L' 'V' 'N' 'L' 'A' 'S' 'Q' 'L' 'L'
 'E' 'N' 'K' 'H' 'H' 'N' 'K' 'W' 'K' 'G' 'I' 'E' 'L' 'Q' 'N' 'N' 'K' 'K'
 'E' 'R' 'E' 'Q' 'K' 'I']
